# **TRABAJO DE FIN DE UNIDAD 02**

# **Estudio comparativo de algoritmos de detección de rostros basados en HOG y CNN con análisis de técnicas de oclusión**

-----


```
Integrantes:
- CHOQUE QUISPE JADYRA CH'ASKA - 204795
- HANCCO CHAMPI FRAN ANTHONY - 204797
- JALLO PACCAYA YASUMY MARICELY - 204799
```



## **2. IMPORTAR DATOS**

Los datos fueron obtenidos del conjunto de datos CelebA a través de la biblioteca `kagglehub`, utilizando el identificador `"jessicali9530/celeba-dataset"`. Este conjunto de datos contiene imágenes de rostros de celebridades y sus respectivas etiquetas de atributos.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/celeba-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.33G/1.33G [00:47<00:00, 30.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/jessicali9530/celeba-dataset/versions/2


**Carga de Imágenes y Atributos del Dataset CelebA**

In [ ]:
import os
import cv2
import pandas as pd

# Ruta a las imágenes y a los atributos
images_path = os.path.join(path, "img_align_celeba")
attributes_path = os.path.join(path, "list_attr_celeba.csv")

# Cargar los atributos de las imágenes
attributes = pd.read_csv(attributes_path)

# Cargar una muestra de imágenes
sample_images = [cv2.imread(os.path.join(images_path, img)) for img in os.listdir(images_path)[:10]]


**Contenido del directorio principal**

In [ ]:
import os

# Listar archivos y carpetas en el directorio principal del dataset
print("Contenido del directorio principal:", os.listdir(path))

# Comprobar si existe la carpeta img_align_celeba
images_folder = os.path.join(path, "img_align_celeba")
if os.path.exists(images_folder):
    print("La carpeta img_align_celeba existe.")
else:
    print("La carpeta img_align_celeba no existe. Verifica el nombre o la estructura de archivos.")


Contenido del directorio principal: ['list_bbox_celeba.csv', 'img_align_celeba', 'list_eval_partition.csv', 'list_attr_celeba.csv', 'list_landmarks_align_celeba.csv']
La carpeta img_align_celeba existe.


In [ ]:
import os

# Imprimir la ruta completa a las imágenes
print("Ruta especificada para images_path:", images_path)

# Listar el contenido de la carpeta principal
print("Contenido de path:", os.listdir(path))

# Listar el contenido de la carpeta img_align_celeba si existe
if os.path.exists(images_path):
    print("Contenido de img_align_celeba:", os.listdir(images_path)[:10])  # Muestra los primeros 10 elementos
else:
    print("La carpeta img_align_celeba no existe en la ubicación especificada.")


Ruta especificada para images_path: /root/.cache/kagglehub/datasets/jessicali9530/celeba-dataset/versions/2/img_align_celeba
Contenido de path: ['list_bbox_celeba.csv', 'img_align_celeba', 'list_eval_partition.csv', 'list_attr_celeba.csv', 'list_landmarks_align_celeba.csv']
Contenido de img_align_celeba: ['img_align_celeba']


**Paso 1. Importar Biblioteca Necesarias**

In [ ]:
# Importar bibliotecas necesarias
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Para el procesamiento de datos
from sklearn.model_selection import train_test_split

# Para el modelo MobileNetV2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Para el modelo ResNet50
from tensorflow.keras.applications import ResNet50

# Para evaluar el rendimiento
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


**Paso 2: Cargar y Preprocesar el Dataset CelebA**

In [ ]:
print(attributes.columns)

Index(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')


In [ ]:
import os
import pandas as pd

# Define paths
images_path = os.path.join(path, "img_align_celeba", "img_align_celeba")
attributes_path = os.path.join(path, "list_attr_celeba.csv")

# Load image attributes
attributes = pd.read_csv(attributes_path)

# Create a function to label occlusions
def label_occlusion(row):
    if row['Eyeglasses'] == 1:
        return "Eyeglasses"
    elif row['Blurry'] == 1:
        return "Blurry"
    elif row['Wearing_Hat'] == 1:
        return "Hat"
    else:
        return "No Occlusion"

# Apply the function to create the 'label' column
attributes['label'] = attributes.apply(label_occlusion, axis=1)

# Show the first records to verify
print(attributes.head())


     image_id  5_o_Clock_Shadow  Arched_Eyebrows  Attractive  Bags_Under_Eyes  \
0  000001.jpg                -1                1           1               -1   
1  000002.jpg                -1               -1          -1                1   
2  000003.jpg                -1               -1          -1               -1   
3  000004.jpg                -1               -1           1               -1   
4  000005.jpg                -1                1           1               -1   

   Bald  Bangs  Big_Lips  Big_Nose  Black_Hair  ...  Smiling  Straight_Hair  \
0    -1     -1        -1        -1          -1  ...        1              1   
1    -1     -1        -1         1          -1  ...        1             -1   
2    -1     -1         1        -1          -1  ...       -1             -1   
3    -1     -1        -1        -1          -1  ...       -1              1   
4    -1     -1         1        -1          -1  ...       -1             -1   

   Wavy_Hair  Wearing_Earrings  Wearin

**Mostrar la columna 'label'**

In [ ]:
# Mostrar los primeros registros para verificar
print(attributes[['image_id', 'label']].sample(20))

          image_id         label
104113  104114.jpg  No Occlusion
94010   094011.jpg  No Occlusion
78755   078756.jpg  No Occlusion
36207   036208.jpg  No Occlusion
140953  140954.jpg  No Occlusion
53958   053959.jpg  No Occlusion
91916   091917.jpg  No Occlusion
124625  124626.jpg  No Occlusion
24687   024688.jpg  No Occlusion
201570  201571.jpg  No Occlusion
80214   080215.jpg  No Occlusion
27170   027171.jpg  No Occlusion
23757   023758.jpg  No Occlusion
106943  106944.jpg  No Occlusion
164324  164325.jpg  No Occlusion
124227  124228.jpg  No Occlusion
171709  171710.jpg           Hat
167415  167416.jpg  No Occlusion
110380  110381.jpg           Hat
37576   037577.jpg    Eyeglasses


### **MODELO MOBILENETV2**

**Dividir el Conjunto de Datos en Entrenamiento y Validación**

In [ ]:
# Dividir el conjunto de datos (80% entrenamiento, 20% validación)
train_df, val_df = np.split(attributes.sample(frac=1, random_state=42), [int(0.8 * len(attributes))])

# Verificar el tamaño de los conjuntos
print(f"Tamaño del conjunto de entrenamiento: {len(train_df)}")
print(f"Tamaño del conjunto de validación: {len(val_df)}")


Tamaño del conjunto de entrenamiento: 162079
Tamaño del conjunto de validación: 40520


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


**Definir y Compilar el Modelo MobileNetV2**

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

def create_mobilenetv2_model():
    # Definir el modelo base
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Descongelar las últimas 20 capas para fine-tuning
    base_model.trainable = True
    for layer in base_model.layers[:-20]:  # Congelar todas menos las últimas 20 capas
        layer.trainable = False

    # Construir el modelo completo
    model = Sequential([
        base_model,
        Flatten(),
        Dropout(0.5),  # Dropout para reducir sobreajuste
        Dense(128, activation='relu'),  # Aumentar el tamaño de esta capa
        Dropout(0.3),  # Dropout adicional
        Dense(6, activation='softmax')  # Cambiar a 6 clases si tienes 6 tipos
    ])

    # Compilar el modelo con el optimizador ajustado
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Crear el modelo con la configuración mejorada
mobilenetv2_model = create_mobilenetv2_model()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.2, patience=3, min_lr=1e-6)
]


#### **Entrenar MobileNetV2 Sin Datos de Oclusiones**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Filtrar solo las imágenes sin oclusiones
train_data_no_occlusion = train_df[train_df['label'] == 'No Occlusion']
val_data_no_occlusion = val_df[val_df['label'] == 'No Occlusion']

# Crear generadores de datos
train_datagen = ImageDataGenerator(rescale=1./255)  # Normalización
val_datagen = ImageDataGenerator(rescale=1./255)

# Crear generadores para el conjunto de entrenamiento y validación
train_generator_no_occlusion = train_datagen.flow_from_dataframe(
    dataframe=train_data_no_occlusion,
    x_col='image_id',
    y_col='label',
    directory=images_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)

val_generator_no_occlusion = val_datagen.flow_from_dataframe(
    dataframe=val_data_no_occlusion,
    x_col='image_id',
    y_col='label',
    directory=images_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)

Found 137697 validated image filenames belonging to 1 classes.
Found 34410 validated image filenames belonging to 1 classes.


In [ ]:
# Entrenar el modelo MobileNetV2 sin datos de oclusión
history_no_occlusion = mobilenetv2_model.fit(
    train_generator_no_occlusion,
    validation_data=val_generator_no_occlusion,
    epochs=5  # Ajusta el número de épocas según sea necesario
)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4304/4304 ━━━━━━━━━━━━━━━━━━━━ 223s 48ms/step - accuracy: 0.9987 - loss: 0.0070 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/5
4304/4304 ━━━━━━━━━━━━━━━━━━━━ 193s 45ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/5
4304/4304 ━━━━━━━━━━━━━━━━━━━━ 193s 45ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/5
4304/4304 ━━━━━━━━━━━━━━━━━━━━ 193s 45ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
4304/4304 ━━━━━━━━━━━━━━━━━━━━ 192s 45ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


Analizar y visualizar el rendimiento del modelo MobileNetV2

#### **Entrenar MobileNetV2 Con Datos que incluyen Oclusiones**

crear y preparar generadores de datos para el entrenamiento y validación de un modelo de detección de rostros con oclusiones.

In [ ]:
# Generadores de datos para el entrenamiento con oclusiones
train_datagen = ImageDataGenerator(rescale=1./255)  # Normalización
val_datagen = ImageDataGenerator(rescale=1./255)

# Crear generadores para el conjunto de entrenamiento y validación con todas las clases de oclusión
train_generator_with_occlusion = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_id',
    y_col='label',
    directory=images_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'  # Configurado para múltiples clases
)

val_generator_with_occlusion = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='image_id',
    y_col='label',
    directory=images_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)

Found 162079 validated image filenames belonging to 4 classes.
Found 40520 validated image filenames belonging to 4 classes.


In [ ]:
# Entrenar el modelo MobileNetV2 con datos que incluyen oclusiones
history_with_occlusion = mobilenetv2_model.fit(
    train_generator_with_occlusion,
    validation_data=val_generator_with_occlusion,
    epochs=10  # Puedes ajustar el número de épocas según sea necesario
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5065/5065 ━━━━━━━━━━━━━━━━━━━━ 238s 47ms/step - accuracy: 0.9241 - loss: 0.4592 - val_accuracy: 0.9461 - val_loss: 0.1514
Epoch 2/10
5065/5065 ━━━━━━━━━━━━━━━━━━━━ 226s 45ms/step - accuracy: 0.9492 - loss: 0.1416 - val_accuracy: 0.9486 - val_loss: 0.1529
Epoch 3/10
5065/5065 ━━━━━━━━━━━━━━━━━━━━ 227s 45ms/step - accuracy: 0.9550 - loss: 0.1222 - val_accuracy: 0.9503 - val_loss: 0.1380
Epoch 4/10
5065/5065 ━━━━━━━━━━━━━━━━━━━━ 230s 45ms/step - accuracy: 0.9592 - loss: 0.1076 - val_accuracy: 0.9412 - val_loss: 0.1597
Epoch 5/10
5065/5065 ━━━━━━━━━━━━━━━━━━━━ 227s 45ms/step - accuracy: 0.9632 - loss: 0.0954 - val_accuracy: 0.9508 - val_loss: 0.1546
Epoch 6/10
5065/5065 ━━━━━━━━━━━━━━━━━━━━ 229s 45ms/step - accuracy: 0.9689 - loss: 0.0794 - val_accuracy: 0.9438 - val_loss: 0.1710
Epoch 7/10
5065/5065 ━━━━━━━━━━━━━━━━━━━━ 230s 45ms/step - accuracy: 0.9746 - loss: 0.0657 - val_accuracy: 0.9390 - val_loss: 0.1967
Epoch 8/10
5065/5065 ━━━━━━━━━━━━━━━━━━━━ 228s 45ms/step - accuracy: 0.9802 - lo

**Evaluar el Modelo MobileNetV2**

In [ ]:
# Evaluar el modelo MobileNetV2 en conjunto de validación con oclusiones
eval_with_occlusion_mobilenet = mobilenetv2_model.evaluate(val_generator_with_occlusion)
print(f"Precisión de MobileNetV2 con oclusiones: {eval_with_occlusion_mobilenet[1]:.4f}")


1267/1267 ━━━━━━━━━━━━━━━━━━━━ 45s 35ms/step - accuracy: 0.9398 - loss: 0.2658
Precisión de MobileNetV2 con oclusiones: 0.9410


In [ ]:
import time

# Medir velocidad para MobileNetV2 en 10 imágenes de validación con oclusiones
start_time = time.time()
for i, (img, label) in enumerate(val_generator_with_occlusion):
    if i >= 10:  # Limitar a 10 imágenes
        break
    mobilenetv2_model.predict(img)
time_taken_mobilenet_with_occlusion = (time.time() - start_time) / 10
print(f"Tiempo promedio de detección de MobileNetV2 con oclusiones: {time_taken_mobilenet_with_occlusion:.4f} segundos")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Tiempo promedio de detección de MobileNetV2 con oclusiones: 0.3536 segundos


Comparar las métricas de entrenamiento y validación de dos modelos MobileNetV2 (uno entrenado con oclusiones y otro sin ellas) y analiza cuál modelo tiene mejor precisión de validación al final del entrenamiento

In [ ]:
# Access training and validation metrics for both models
train_accuracy_no_occlusion = history_no_occlusion.history['accuracy']
val_accuracy_no_occlusion = history_no_occlusion.history['val_accuracy']
train_loss_no_occlusion = history_no_occlusion.history['loss']
val_loss_no_occlusion = history_no_occlusion.history['val_loss']

train_accuracy_with_occlusion = history_with_occlusion.history['accuracy']
val_accuracy_with_occlusion = history_with_occlusion.history['val_accuracy']
train_loss_with_occlusion = history_with_occlusion.history['loss']
val_loss_with_occlusion = history_with_occlusion.history['val_loss']

# Print the results
print("MobileNetV2 without Occlusion Data:")
print("Training Accuracy:", train_accuracy_no_occlusion)
print("Validation Accuracy:", val_accuracy_no_occlusion)
print("Training Loss:", train_loss_no_occlusion)
print("Validation Loss:", val_loss_no_occlusion)


print("\nMobileNetV2 with Occlusion Data:")
print("Training Accuracy:", train_accuracy_with_occlusion)
print("Validation Accuracy:", val_accuracy_with_occlusion)
print("Training Loss:", train_loss_with_occlusion)
print("Validation Loss:", val_loss_with_occlusion)

# Comparison:
print("\nComparación de Resultados:")

# Compare final validation accuracies
final_accuracy_no_occlusion = val_accuracy_no_occlusion[-1]
final_accuracy_with_occlusion = val_accuracy_with_occlusion[-1]

print(f"- Precisión de validación final (sin oclusiones): {final_accuracy_no_occlusion:.4f}")
print(f"- Precisión de validación final (con oclusiones): {final_accuracy_with_occlusion:.4f}")

if final_accuracy_no_occlusion > final_accuracy_with_occlusion:
    print("- El modelo entrenado sin datos de oclusión obtuvo una precisión de validación mayor.")
elif final_accuracy_with_occlusion > final_accuracy_no_occlusion:
    print("- El modelo entrenado con datos de oclusión obtuvo una precisión de validación mayor.")
else:
    print("- Ambos modelos obtuvieron la misma precisión de validación.")

MobileNetV2 without Occlusion Data:
Training Accuracy: [0.9998547434806824, 1.0, 1.0, 1.0, 1.0]
Validation Accuracy: [1.0, 1.0, 1.0, 1.0, 1.0]
Training Loss: [0.0007825368666090071, 0.0, 0.0, 0.0, 0.0]
Validation Loss: [0.0, 0.0, 0.0, 0.0, 0.0]

MobileNetV2 with Occlusion Data:
Training Accuracy: [0.938178300857544, 0.9498269557952881, 0.9540162682533264, 0.9589089155197144, 0.9625552892684937, 0.9673554301261902, 0.9732291102409363, 0.9786708950996399, 0.9819470643997192, 0.9852602481842041]
Validation Accuracy: [0.9461007118225098, 0.94856858253479, 0.950345516204834, 0.9412142038345337, 0.950765073299408, 0.9437808394432068, 0.9390177726745605, 0.9457551836967468, 0.9481737613677979, 0.9410414695739746]
Training Loss: [0.21774154901504517, 0.1427031010389328, 0.12529656291007996, 0.10942304879426956, 0.09754703938961029, 0.0828733742237091, 0.0689094290137291, 0.05764495208859444, 0.04833448678255081, 0.04111591726541519]
Validation Loss: [0.15140192210674286, 0.1529390811920166, 0.

Comparar la precisión de entrenamiento y validación de dos modelos MobileNetV2 (uno entrenado con oclusiones y otro sin ellas)

 Función para predecir la clase de una imagen utilizando un modelo MobileNetV2, que clasifica entre "Blurry," "Eyeglasses," "Hat" y "No Occlusion"

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Función de predicción para la clasificación de imagen con todas las clases
def predict_image_class(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.  # Normalizar la imagen

    prediction = mobilenetv2_model.predict(img_array)
    predicted_class = np.argmax(prediction)

    # Actualizar con las etiquetas de clase
    class_labels = {
        0: 'Blurry',
        1: 'Eyeglasses',
        2: 'Hat',
        3: 'No Occlusion'
    }
    predicted_label = class_labels.get(predicted_class, "Clase desconocida")

    print(f"Predicción para la imagen es: {predicted_label}")
    return predicted_label

# Ejemplo de uso (reemplazar con tu ruta de imagen)
image_path = '/root/.cache/kagglehub/datasets/jessicali9530/celeba-dataset/versions/2/img_align_celeba/img_align_celeba/000001.jpg' # Reemplazar con la ruta de tu imagen
predicted_label = predict_image_class(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicción para la imagen es: No Occlusion


Convertir un modelo de Keras (en este caso, un modelo de MobileNetV2) a un formato TensorFlow Lite.
Esto es esencial para facilitar el uso del modelo en dispositivos móviles donde se requiere un tamaño de modelo más pequeño y un tiempo de inferencia más rápido.

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(mobilenetv2_model)
tflite_model = converter.convert()

with open("model.tflite", 'wb') as f:
  f.write(tflite_model)

Saved artifact at '/tmp/tmpe4gytone'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  134786386317856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134783203515504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134783203511280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134783203511632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134783203522544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134783188636368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134783188636016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134783188644112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134783188633552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134783188642176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1347831886